In [25]:
import pandas as pd
import os
import numpy as np
import requests
from bs4 import BeautifulSoup
import bs4.element
import warnings
warnings.filterwarnings('ignore')
import re
import time
from datetime import datetime

In [26]:
import pymysql
conn = pymysql.connect(
    host='34.64.240.96'
    , user='root'
    , password='tndusWkd1.'
    , db='final_project'
    , charset='utf8'
)
cur = conn.cursor()

In [27]:
def clean_text(d):
  text = re.sub(r'\([^)]*\)', '', str(d))
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\<[^>]*\>', '', text)
  pattern = r'[^가-힣0-9a-zA-Z\s]'
  text = re.sub(pattern, ' ', text)
  text = re.sub(r'사진', ' ', text)
  text = re.sub(r'.*뉴스', ' ', text)
  text = re.sub("\n", ' ', text)
  text = re.sub("  +", " ", text)
  return text

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\']'           
    text = re.sub(pattern, '', text)
    pattern = '[\"]'            
    text = re.sub(pattern, '', text)
    return text  

In [28]:
folder = '코스피200'

In [29]:
folders = os.listdir('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}'.format(folder))

df_all = pd.DataFrame()
for i in range(0,len(folders)):
    if folders[i].split('.')[1] == 'csv':
        file = 'D:\\big16\\final project\\DATA\\NEWS_DATA\\{}\\'.format(folder)+folders[i]
        df= pd.read_csv(file,encoding='utf-8') 
        df_all = pd.concat([df_all, df])

df_all

,date,title,linkcontent,link,content
0,2021-08-31 12:23:01,"[영상] '전쟁 올림픽' 참가한 중국, 주력 전투기로 군사력 과시",NaN,https://n.news.naver.com/mnews/article/001/001...,"[\n(서울=연합뉴스) 중국이 러시아, 이란, 베네수엘라 등이 참가하는 국제 군사 ..."
1,2021-08-29 07:01:17,4년째 전투기 개발 분담금 안 내는 인도네시아… 끝까지 같이 가려는 軍,NaN,https://n.news.naver.com/mnews/article/366/000...,[\n\t\t\t 최근 인도네시아가 한국형 전투기 KF-21 공동연구개...
2,2021-08-29 06:03:06,"태평양에 쏠린 눈, 중국과 인도는 왜 항모를 만드는가 [박수찬의 軍]",NaN,https://n.news.naver.com/mnews/article/022/000...,[\n \n\n\n\n 중국 해군 항모 랴오닝호가 훈련을 위해 항해를 하고 있다...
3,2021-08-28 18:26:07,'빨갱이' 동지는 다 죽었는데 혼자 살아남은 남자 [박만순의 기억전쟁2],NaN,https://n.news.naver.com/mnews/article/047/000...,"[\n영동군 양산면 호탄리 주민들의 비극... 보도연맹으로 청년들 떼죽음""김철민을 ..."
4,2021-08-26 17:07:17,"대만 내년 국방예산 19.2조원 5.2%↑...""전투기 도입에 1.68조원""",NaN,https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n[서울=뉴시스]미국재대만협회(AIT)는 27일 페이스북에 미국 ...
...,...,...,...,...,...
5,2023-01-31 16:28:55,日 전투기 탑승한 나토 사무총장,NaN,https://n.news.naver.com/mnews/article/421/000...,[\n\n\n\n\n(사야마 AFP=뉴스1) 김성식 기자 = 31일 일본 사이타마현...
6,2023-01-31 16:52:29,"日기시다, 전투기 공격시에도 반격능력 대응 배제 안해",NaN,https://n.news.naver.com/mnews/article/003/001...,"[\n기사내용 요약""엄밀히 생각한 후 대응""\n\n\n\n[워싱턴DC=AP/뉴시스]..."
7,2023-01-31 17:21:47,"[영상] ""탱크 9천만·전투기 2억6천만원""…러 기업, 서방 무기에 현상금",NaN,https://n.news.naver.com/mnews/article/001/001...,[\n(서울=연합뉴스) 서방이 우크라이나에 대한 전차 지원안을 발표하자 러시아 기업...
8,2023-01-31 13:31:02,"우크라 이번엔 전투기 요구…바이든, 일단 ""NO""",NaN,https://n.news.naver.com/mnews/article/422/000...,[\n\t\t\t[앵커]서방의 탱크 지원이 결정되자 우크라이나는 이제 전투기 지원을...


In [30]:
df_all.drop('link', axis=1, inplace=True)
# df_all.drop('Unnamed: 0', axis=1, inplace=True)
df_all.drop('linkcontent', axis=1, inplace=True)

In [31]:
df_all.dropna(axis=0, inplace=True)
df_all

,date,title,content
0,2021-08-31 12:23:01,"[영상] '전쟁 올림픽' 참가한 중국, 주력 전투기로 군사력 과시","[\n(서울=연합뉴스) 중국이 러시아, 이란, 베네수엘라 등이 참가하는 국제 군사 ..."
1,2021-08-29 07:01:17,4년째 전투기 개발 분담금 안 내는 인도네시아… 끝까지 같이 가려는 軍,[\n\t\t\t 최근 인도네시아가 한국형 전투기 KF-21 공동연구개...
2,2021-08-29 06:03:06,"태평양에 쏠린 눈, 중국과 인도는 왜 항모를 만드는가 [박수찬의 軍]",[\n \n\n\n\n 중국 해군 항모 랴오닝호가 훈련을 위해 항해를 하고 있다...
3,2021-08-28 18:26:07,'빨갱이' 동지는 다 죽었는데 혼자 살아남은 남자 [박만순의 기억전쟁2],"[\n영동군 양산면 호탄리 주민들의 비극... 보도연맹으로 청년들 떼죽음""김철민을 ..."
4,2021-08-26 17:07:17,"대만 내년 국방예산 19.2조원 5.2%↑...""전투기 도입에 1.68조원""",[\n\n\n\n\n[서울=뉴시스]미국재대만협회(AIT)는 27일 페이스북에 미국 ...
...,...,...,...
5,2023-01-31 16:28:55,日 전투기 탑승한 나토 사무총장,[\n\n\n\n\n(사야마 AFP=뉴스1) 김성식 기자 = 31일 일본 사이타마현...
6,2023-01-31 16:52:29,"日기시다, 전투기 공격시에도 반격능력 대응 배제 안해","[\n기사내용 요약""엄밀히 생각한 후 대응""\n\n\n\n[워싱턴DC=AP/뉴시스]..."
7,2023-01-31 17:21:47,"[영상] ""탱크 9천만·전투기 2억6천만원""…러 기업, 서방 무기에 현상금",[\n(서울=연합뉴스) 서방이 우크라이나에 대한 전차 지원안을 발표하자 러시아 기업...
8,2023-01-31 13:31:02,"우크라 이번엔 전투기 요구…바이든, 일단 ""NO""",[\n\t\t\t[앵커]서방의 탱크 지원이 결정되자 우크라이나는 이제 전투기 지원을...


In [32]:
df_all['content'] = df_all['content'].apply(text_clean)
df_all['content'] = df_all['content'].apply(clean_text)
df_all['title'] = df_all['title'].apply(text_clean)
df_all['title'] = df_all['title'].apply(clean_text)
df_all

,date,title,content
0,2021-08-31 12:23:01,영상 전쟁 올림픽 참가한 중국 주력 전투기로 군사력 과시,중국이 러시아 이란 베네수엘라 등이 참가하는 국제 군사 경연에 주력 전투기와 병력...
1,2021-08-29 07:01:17,년째 전투기 개발 분담금 안 내는 인도네시아 끝까지 같이 가려는,공동개발이 지지부진한 사이 경쟁국인 프랑스의 움직임은 빨라지고 있다 인도네시아가 ...
2,2021-08-29 06:03:06,태평양에 쏠린 눈 중국과 인도는 왜 항모를 만드는가 박수찬의,이후 인도는 경제적 문제가 발생해도 항모 전력은 우선순위를 두고 유지해왔다 인도가...
3,2021-08-28 18:26:07,빨갱이 동지는 다 죽었는데 혼자 살아남은 남자 박만순의 기억전쟁,영동군 양산면 호탄리 주민들의 비극 보도연맹으로 청년들 떼죽음김철민을 잡았다 와하며...
4,2021-08-26 17:07:17,대만 내년 국방예산 조원 전투기 도입에 조원,서울뉴시스미국재대만협회는 일 페이스북에 미국 애리조나주 루크 공군기지에서 공중 급유...
...,...,...,...
5,2023-01-31 16:28:55,전투기 탑승한 나토 사무총장,
6,2023-01-31 16:52:29,기시다 전투기 공격시에도 반격능력 대응 배제 안해,기사내용 요약엄밀히 생각한 후 대응워싱턴뉴시스지난 일현지시간 미국 워싱턴 백악관에서...
7,2023-01-31 17:21:47,영상 탱크 천만전투기 억천만원러 기업 서방 무기에 현상금,등이 일현지시간 보도했습니다 러시아 에너지 관련 기업 포레스는 지난 일 웹사이트를...
8,2023-01-31 13:31:02,우크라 이번엔 전투기 요구바이든 일단,기사문의 및 제보 카톡라인


In [33]:
df_all.dropna(axis=0, inplace=True)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13350 entries, 0 to 9
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     13350 non-null  object
 1   title    13350 non-null  object
 2   content  13350 non-null  object
dtypes: object(3)
memory usage: 417.2+ KB


In [34]:
df_all.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\0. merge\\{}_merge.csv'.format(folder), index=False)

In [35]:
NewsDate = df_all['date'].to_list()
NewsTItle = df_all['title'].to_list()
NewsContent = df_all['content'].to_list()

In [36]:
for date,title,content in zip(NewsDate, NewsTItle, NewsContent):
    sql = "INSERT INTO past_news_fighterplane VALUES ({}, {}, {})".format("\""+date+"\"", "\""+title+"\"", "\""+content+"\"")
    cur.execute(sql)
conn.commit()
conn.close()